### Haneen Hisham 20236032
### Bonus Lab 5 Task

In [2]:
# install required libraries
!pip install dash
!pip install dash-html-components
!pip install dash-core-components
!pip install dash_bootstrap_components
!pip install dash_bootstrap_templates
!pip install gapminder
!pip install plotly

   ---------------------------------------- 0.0/7.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.9 MB ? eta -:--:--
   -- ------------------------------------- 0.5/7.9 MB 1.9 MB/s eta 0:00:04
   ----- ---------------------------------- 1.0/7.9 MB 1.9 MB/s eta 0:00:04
   ------ --------------------------------- 1.3/7.9 MB 1.9 MB/s eta 0:00:04
   --------- ------------------------------ 1.8/7.9 MB 1.9 MB/s eta 0:00:04
   ---------- ----------------------------- 2.1/7.9 MB 1.9 MB/s eta 0:00:04
   ------------- -------------------------- 2.6/7.9 MB 1.9 MB/s eta 0:00:03
   -------------- ------------------------- 2.9/7.9 MB 1.9 MB/s eta 0:00:03
   ----------------- ---------------------- 3.4/7.9 MB 1.9 MB/s eta 0:00:03
   ------------------ --------------------- 3.7/7.9 MB 1.8 MB/s eta 0:00:03
   ------------------- -------------------- 3.9/7.9 MB 1.8 MB/s eta 0:00:03
   --------------------- ------------------ 4.2/7.9 MB 1.8 MB/s eta 0:00:03
   -----------------------

In [1]:
from dash import Dash, html, dcc, dash_table, Input, Output, callback
import dash_bootstrap_components as dbc
import plotly.express as px
from dash_bootstrap_templates import ThemeChangerAIO, template_from_url


# load dataset once to avoid reloading it on every request
gap = px.data.gapminder()

#store unique years & continents for UI inputs
year_list = sorted(gap["year"].unique())
continent_list = sorted(gap["continent"].unique())


# initialize app with bootstrap + template styles
external_styles = [
    dbc.themes.BOOTSTRAP,
    "https://cdn.jsdelivr.net/gh/AnnMarieW/dash-bootstrap-templates/dbc.min.css",
]
app = Dash(__name__, external_stylesheets=external_styles)

# -------------------------------------------------------
# header of the page
# -------------------------------------------------------
header_section = html.H4(
    "Interactive Gapminder Dashboard",
    className="bg-primary text-center text-white p-2 mb-3",
)


# data table component (used in table tab)
data_table = html.Div(
    dash_table.DataTable(
        id="gap-table",
        columns=[{"name": col, "id": col} for col in gap.columns],  # columns from dataset
        data=gap.to_dict("records"),  # initial table contents
        editable=True,
        filter_action="native",
        sort_action="native",
        page_size=10,
        style_table={"overflowX": "auto"},  # allow horizontal scroll
        row_selectable="multi",
    ),
    className="dbc-row-selectable",
)


# dropdown for y-axis indicator (gdpPercap, lifeExp, pop)
indicator_dropdown = html.Div(
    [
        dbc.Label("Y-Axis Indicator"),
        dcc.Dropdown(
            id="y-indicator",
            options=[{"label": x, "value": x} for x in ["gdpPercap", "lifeExp", "pop"]],
            value="pop",  # default indicator
            clearable=False,
        ),
    ],
    className="mb-3",
)


# checklist of continents (user can toggle which continents appear)
continent_selector = html.Div(
    [
        dbc.Label("Choose Continents"),
        dbc.Checklist(
            id="continent-selection",
            options=[{"label": c, "value": c} for c in continent_list],
            value=continent_list,  # select all by default
            inline=True,
        ),
    ],
    className="mb-3",
)

# year slider with proper marks 
year_slider = html.Div(
    [
        dbc.Label("Year Range"),
        dcc.RangeSlider(
            id="year-slider",
            min=int(year_list[0]),
            max=int(year_list[-1]),
            step=5,
            value=[int(year_list[2]), int(year_list[-2])],

           # only show first and last year so UI is clean
            marks={
                str(year_list[0]): str(year_list[0]),
                str(year_list[-1]): str(year_list[-1]),
            },

            tooltip={"always_visible": True},
        ),
    ],
    className="mb-3",
)



# combine all controls into one card
control_panel = dbc.Card(
    [
        indicator_dropdown,
        continent_selector,
        year_slider,
        ThemeChangerAIO(aio_id="theme"),  # theme switcher
    ],
    body=True,
)


# graph tabs
tab_line = dbc.Tab(dcc.Graph(id="line-plot"), label="Line Chart")
tab_scatter = dbc.Tab(dcc.Graph(id="scatter-plot"), label="Scatter Chart")
tab_table = dbc.Tab(data_table, label="Table", className="p-3")

#wrap all tabs in a card
tab_container = dbc.Card(dbc.Tabs([tab_line, tab_scatter, tab_table]))


# main layout of the app
app.layout = dbc.Container(
    [
        header_section,
        dbc.Row(
            [
                dbc.Col(control_panel, width=4),  # left side controls
                dbc.Col(tab_container, width=8),   # right side output
            ]
        ),
    ],
    fluid=True,
)

# callback: main logic to update line graph, scatter graph, and table
@callback(
    Output("line-plot", "figure"),
    Output("scatter-plot", "figure"),
    Output("gap-table", "data"),
    Input("y-indicator", "value"),
    Input("continent-selection", "value"),
    Input("year-slider", "value"),
    Input(ThemeChangerAIO.ids.radio("theme"), "value"),
)
def update_visuals(y_value, chosen_continents, year_range, theme_selected):

    #empty selection case
    if not chosen_continents:
        return {}, {}, []

    #filter based on user-selected years & continents
    filtered = gap[
        (gap["year"] >= year_range[0])
        & (gap["year"] <= year_range[1])
        & (gap["continent"].isin(chosen_continents))
    ]

    #table needs the filtered data
    table_data = filtered.to_dict("records")

    # line chart
    fig_line = px.line(
        filtered,
        x="year",
        y=y_value,
        color="continent",
        line_group="country",
        template=template_from_url(theme_selected),  # theme applied
    )

    # scatter chart
    latest_year = year_range[1]  # show scatter for latest selected year
    scatter_data = filtered[filtered["year"] == latest_year]

    fig_scat = px.scatter(
        scatter_data,
        x="gdpPercap",
        y="lifeExp",
        size="pop",
        color="continent",
        log_x=True,
        size_max=60,
        template=template_from_url(theme_selected),
        title=f"Gapminder {latest_year} — {theme_selected} Theme",
    )

    return fig_line, fig_scat, table_data

# run server
if __name__ == "__main__":
    
    app.run(debug=False, use_reloader=False, port=8051)
    